In [ ]:
import sys
import os
sys.path.append(os.path.abspath("../../"))
import torch
from torchvision import transforms
from PIL import Image
import os
import numpy as np
import cv2
from models.pix2pix_model import Pix2PixModel
from options.base_options import BaseOptions
from data.produce_dataset import Pix2PixDataset
from torch.utils.data import Dataset, DataLoader 
import time
from gans import Training, Options
from metrics import compute_metric
from infer import Infer
import matplotlib.pyplot as plt

# Pix2Pix: Gen(image) = image

In [ ]:
image_dir = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\unified_set\\images"
label_dir = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\unified_set\\labels"

# Create dataset and dataloader
dataset = Pix2PixDataset(image_dir, label_dir)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4)

print(f"Loaded dataset with {len(dataset)} image-mask pairs.")

In [ ]:
input_dir = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\transformed_images_labels\\images"   # Raw cell images
mask_dir = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\preprocessing_outputs\\transformed_images_labels\\labels"     # Corresponding segmentation masks
input_tuning_dir = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\tuning\\transformed_images_labels\\images"
output_dir = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\Training-unlabeled\\Training-unlabeled\\labels" # Output paired images
unlabeled_images_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\transformed_images_labels\\images"  # Unlabeled images
pseudo_mask_output_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\dataset_pix2pix\\test\\generated_masks"  # Where to save masks
pseudo_mask_output_tuning_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\dataset_pix2pix\\tuning\\generated_masks_test"

options = Options()
training = Training()
infer = Infer()

opt = options.get_opt()
pix2pix_model = training.load_pix2pix_model(opt)

# Train pix2pix 
training.train_model(pix2pix_model, dataset, opt)

pix2pix_model = Pix2PixModel(opt)
checkpoint_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\src\\data_augmentation\\models\\pix_2_pix_bigger_corpus\\latest_generator.pth"

# Load state dictionary
#pix2pix_model.load_networks("latest")
pix2pix_model.netG.load_state_dict(torch.load(checkpoint_path))

pix2pix_model.netG.eval()
print("Model loaded successfully!")

# Generate pseudo-masks
infer.generate_pseudo_masks(pix2pix_model, input_tuning_dir, pseudo_mask_output_tuning_path)
print("Synthetic pseudo-masks added to training dataset!")


# Run metrics for pix2pix model

In [ ]:
gt_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\tuning\\transformed_images_labels\\labels"
pred_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\dataset_pix2pix\\tuning\\generated_masks_test"

# Simulate command-line arguments
sys.argv = [
    "compute_metric.py",  # Script name (can be anything)
    "-g", gt_path,  # Replace with the actual ground truth path
    "-s", pred_path,  # Replace with the actual segmentation results path
    "--gt_suffix", ".png",
    "--seg_suffix", ".png",
    "-thre", "0.5", "0.6",  # Example thresholds
    "-o", "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\output",  # Output path
    "-n", "results"  # Output file name
]

# Call the main function
compute_metric.main()